In [1]:
#| default_exp models.upma

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [165]:
#| export
import torch, torch.nn as nn, re, os, numpy as np, gc, torch.nn.functional as F
from tqdm.auto import tqdm
from dataclasses import dataclass
from torch.nn.parallel import DataParallel
from torch.utils.data import DataLoader
from typing import Optional, Union, Tuple, Any, Dict, Sequence, List

from transformers import DistilBertConfig, DistilBertModel, PretrainedConfig, PreTrainedModel
from transformers.modeling_outputs import BaseModelOutput, ModelOutput
from transformers.models.distilbert.modeling_distilbert import Embeddings, TransformerBlock, create_sinusoidal_embeddings
from transformers.activations import get_activation
from transformers.modeling_attn_mask_utils import _prepare_4d_attention_mask_for_sdpa
from transformers.pytorch_utils import apply_chunking_to_forward

from xcai.core import *
from xcai.losses import *
from xcai.data import MainXCDataset
from xcai.sdata import SMainXCDataset, identity_collate_fn
from xcai.learner import XCDataParallel
from xcai.models.modeling_utils import Pooling

## Load data

In [5]:
from xcai.main import *

In [6]:
data_dir = '/Users/suchith720/Projects/data'
config_file = 'wikiseealsotitles'
config_key = 'data_meta'

mname = 'sentence-transformers/msmarco-distilbert-base-v4'

In [7]:
pkl_dir = f'{data_dir}/processed/mogicX/'
pkl_file = f'{pkl_dir}/wikiseealsotitles_data-meta_distilbert-base-uncased_sxc.joblib'

In [8]:
block = build_block(pkl_file, config_file, True, config_key, data_dir=data_dir, 
                    n_sdata_meta_samples=3, do_build=False)

In [9]:
batch = block.train.dset.__getitems__([10, 30])
batch = block.train.dset.__getitems__([10, 200, 500])

In [57]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_scores', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_scores', 'cat2data_data2ptr', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'pcat2lbl_idx', 'pcat2lbl_lbl2ptr', 'cat2lbl_idx', 'cat2lbl_scores', 'cat2lbl_lbl2ptr', 'cat2lbl_identifier', 'cat2lbl_input_text', 'cat2lbl_input_ids', 'cat2lbl_attention_mask', 'cat2lbl_data2ptr', 'pcat2lbl_data2ptr'])

## Configuration

In [154]:
#| export
class UPMAConfig(DistilBertConfig):

    def __init__(
        self,
        memory_module_names: Optional[Union[str, List[str]]] = "embeddings",
        memory_injection_layers: Optional[Union[int, List[int]]] = None,
        
        num_total_metadata: Optional[int] = None,
        num_input_metadata: Optional[int] = 3,
        metadata_dropout: Optional[float] = 0.1,
        
        n_memory_layers: Optional[int] = 3,
        
        data_aug_meta_prefix: Optional[str] = None, 
        lbl2data_aug_meta_prefix: Optional[str] = None,
        neg2data_aug_meta_prefix: Optional[str] = None,

        data_inject_memory: Optional[bool] = True,
        lbl2data_inject_memory: Optional[bool] = True,
        neg2data_inject_memory: Optional[bool] = True,
        
        data_repr_pooling: Optional[bool] = True,
        data_normalize: Optional[bool] = False,

        margin: Optional[float] = 0.3,
        num_negatives: Optional[int] = 10,
        tau: Optional[float] = 0.1,
        apply_softmax: Optional[bool] = True,
        reduction: Optional[str] = "mean",

        calib_margin: Optional[float] = 0.05,
        calib_num_negatives: Optional[int] = 10,
        calib_tau: Optional[float] = 0.1,
        calib_apply_softmax: Optional[bool] = False,
        
        calib_loss_weight: Optional[float] = 0.1,
        use_calib_loss: Optional[bool] = False,

        use_encoder_parallel: Optional[bool] = False,

        loss_function: Optional[str] = "triplet",
        initialize_memory_embeddings_from_injection_layer_mean: Optional[bool] = True,
        metadata_embedding_file: Optional[str] = None,
        
        **kwargs,
    ):
        store_attr('memory_module_names,memory_injection_layers')
        store_attr('num_total_metadata,num_input_metadata,metadata_dropout')
        store_attr('n_memory_layers')
        store_attr('data_aug_meta_prefix,lbl2data_aug_meta_prefix,neg2data_aug_meta_prefix')
        store_attr('data_inject_memory,lbl2data_inject_memory,neg2data_inject_memory')
        store_attr('data_repr_pooling,data_normalize')
        store_attr('margin,num_negatives,tau,apply_softmax,reduction')
        store_attr('calib_margin,calib_num_negatives,calib_tau,calib_apply_softmax')
        store_attr('calib_loss_weight,use_calib_loss,use_encoder_parallel')
        store_attr('loss_function,initialize_memory_embeddings_from_injection_layer_mean,metadata_embedding_file')
        
        super().__init__(**kwargs)
        
        if isinstance(memory_module_names, str): self.memory_module_names = [memory_module_names]
        if memory_injection_layers is None: 
            self.memory_injection_layers = [self.n_layers] * len(self.memory_module_names)

        assert len(self.memory_injection_layers) == len(self.memory_module_names), (
            f"Mismatch in configuration: expected equal lengths, but got "
            f"{len(self.memory_injection_layers)} injection layers and "
            f"{len(self.memory_module_names)} module names."
        )

        idx = np.argsort(self.memory_injection_layers)
        self.memory_injection_layers = [self.memory_injection_layers[i] for i in idx]
        self.memory_module_names = [self.memory_module_names[i] for i in idx]
        
        assert np.all([n <= self.n_layers and n > 0 for n in self.memory_injection_layers]), (
            f"Invalid memory injection layer: {self.memory_injection_layers}. "
            f"It must be less than or equal to the total number of layers ({self.n_layers}) and greater than zero."
        )
        

### Example

In [155]:
config = UPMAConfig(
    memory_module_names = ["embeddings", "encoder"],
    memory_injection_layers = [3, 1],
    
    num_total_metadata = block.train.dset.meta['cat_meta'].n_meta,
    num_input_metadata = 3,
    metadata_dropout = 0.1,
    
    n_memory_layers = 3,

    data_aug_meta_prefix="cat2data",
    lbl2data_aug_meta_prefix="cat2lbl",
    neg2data_aug_meta_prefix="cat2neg",

    data_inject_memory=True,
    lbl2data_inject_memory=False,
    neg2data_inject_memory=False,
    
    data_repr_pooling=False,
    data_normalize=False,

    margin=0.3,
    num_negatives=5,
    tau=0.1,
    apply_softmax=True,
    reduction="mean",

    calib_margin=0.3,
    calib_num_negatives=10,
    calib_tau=0.1,
    calib_apply_softmax=False,
    
    calib_loss_weight=0.1,
    use_calib_loss=True,
    
    use_encoder_parallel=False,
    loss_function="triplet",
)

In [156]:
config.data_inject_memory, config.lbl2data_inject_memory

(True, False)

In [157]:
config.memory_injection_layers, config.memory_module_names

([1, 3], ['encoder', 'embeddings'])

In [160]:
config.loss_function, config.reduction, config.data_normalize

('triplet', 'mean', False)

## Helper functions

In [59]:
#| export
def get_memory_module(name: str):
    if name == "embeddings": return UPMAEmbeddingMemory
    elif name == "encoder": return UPMAEncoderMemory
    else: raise ValueError(f"Invalid memory module: {name}")

def get_loss_function(name:str):
    if name == "triplet": return MultiTripletWithNegatives
    elif name == "ranking": return MultiRankingWithNegatives
    elif name == "soupcon": return MultiSoupConWithNegatives
    elif name == "margin": return MarginMSEWithNegatives
    else:
        raise ValueError(
            f"Invalid loss function name: '{name}'. "
            "Please provide a valid loss function identifier supported by this module."
        )

def align_tensor(tensor:torch.Tensor, indptr:torch.Tensor, pad_tok:Optional[Union[int,float]]=0):
    tensor_shape = tensor.shape
    r, c = len(indptr), indptr.max()

    row_idx = torch.repeat_interleave(torch.arange(r, device=tensor.device), indptr)
    indptr = torch.cat([indptr.new_tensor([0]), indptr.cumsum(dim=0)[:-1]], dim=0)
    within_idx = torch.arange(tensor_shape[0], device=indptr.device) - indptr[row_idx]

    output = torch.full((r, c, *tensor_shape[1:]), pad_tok, device=tensor.device, dtype=tensor.dtype)
    mask = torch.zeros((r, c), device=tensor.device)

    output[row_idx, within_idx] = tensor
    mask[row_idx, within_idx] = 1.0

    return output, mask

def alignment_mask(indptr:torch.Tensor):
    n, r, c = indptr.sum(), len(indptr), indptr.max()

    row_idx = torch.repeat_interleave(torch.arange(r, device=indptr.device), indptr)
    indptr = torch.cat([indptr.new_tensor([0]), indptr.cumsum(dim=0)[:-1]], dim=0)
    within_idx = torch.arange(n, device=indptr.device) - indptr[row_idx]

    mask = torch.zeros((r, c), device=indptr.device, dtype=torch.int64)
    mask[row_idx, within_idx] = 1
    return mask


### Examples

In [60]:
t = torch.randint(100, size=(5, 3))
indptr = torch.tensor([1, 1, 0, 3])

In [61]:
o = align_tensor(t, indptr)

In [62]:
o

(tensor([[[ 6, 84, 31],
          [ 0,  0,  0],
          [ 0,  0,  0]],
 
         [[16, 12, 85],
          [ 0,  0,  0],
          [ 0,  0,  0]],
 
         [[ 0,  0,  0],
          [ 0,  0,  0],
          [ 0,  0,  0]],
 
         [[95, 29, 80],
          [82, 45, 10],
          [81, 43, 63]]]),
 tensor([[1., 0., 0.],
         [1., 0., 0.],
         [0., 0., 0.],
         [1., 1., 1.]]))

## Outputs

In [48]:
#| export
@dataclass
class UPMAEncoderOutput(BaseModelOutput):
    repr: Optional[torch.FloatTensor] = None
    
@dataclass
class UPMAModelOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    data_repr: Optional[torch.FloatTensor] = None
    lbl2data_repr: Optional[torch.FloatTensor] = None
    neg2data_repr: Optional[torch.FloatTensor] = None
    

## Modeling

### `FFN`

* $\hat{x} = dropout(W_2 * max(0, W_1 * x))$

In [17]:
#| export
class FFN(nn.Module):
    def __init__(
        self, 
        config:PretrainedConfig,
        input_dim:int,
        hidden_dim:int,
        output_dim:int,
    ):
        super().__init__()
        self.config = config
        self.dropout = nn.Dropout(p=config.dropout)
        self.chunk_size_feed_forward = config.chunk_size_feed_forward
        self.seq_len_dim = 1
        self.lin1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.lin2 = nn.Linear(in_features=hidden_dim, out_features=output_dim)
        self.activation = get_activation(config.activation)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return apply_chunking_to_forward(self.ff_chunk, self.chunk_size_feed_forward, self.seq_len_dim, input)

    def ff_chunk(self, input: torch.Tensor) -> torch.Tensor:
        x = self.lin1(input)
        x = self.activation(x)
        x = self.lin2(x)
        x = self.dropout(x)
        return x
        

### `Embedding Memory`

* $\hat{\mathcal{A}} = \{a_1, a_2, a_3, \dots a_n\}$ be relevant metadata predicted by the linker.

* $\hat{\mathcal{S}} = \{s_1, s_2, s_3, \dots s_n\}$ be scores of the predicted metadata.

* $\hat{\mathcal{R}} = \{1, 2, 3, \dots n\}$ be rank for the predicted metadata.

* $\mathcal{K} \in R^{M \times D}$ be the $M$ memory items for each metadata.

* $\mathcal{P} \in R^{N \times D}$ be the $N$ positional embeddings.

* Rank and score aware metadata representation: $x_m = \mathcal{K}(\hat{\mathcal{A}}) + MLP(\hat{\mathcal{S}}) + \mathcal{P}(\hat{\mathcal{R}})$

In [128]:
#| export
class UPMAEmbeddingMemory(nn.Module):
    
    def __init__(
        self, 
        config: PretrainedConfig
    ):
        super().__init__()
        self.config = config
        self.metadata_embeddings = nn.Embedding(config.num_total_metadata+1, config.dim, padding_idx=config.num_total_metadata)
        self.rank_embeddings = nn.Embedding(config.num_input_metadata, config.dim)
        
        self.score_ffn = FFN(config, input_dim=1, hidden_dim=config.hidden_dim, output_dim=config.dim)
        self.out_ffn = FFN(config, input_dim=config.dim, hidden_dim=config.hidden_dim, output_dim=config.dim)

        self.LayerNorm = nn.LayerNorm(config.dim, eps=1e-12)
        self.dropout = nn.Dropout(config.metadata_dropout)
        self.register_buffer(
            "position_ids", torch.arange(config.num_input_metadata).expand((1, -1)), persistent=False
        )

    @classmethod
    def from_pretrained(
        cls, 
        config:PretrainedConfig
    ):
        model = cls(config)
        model.eval()
        return model
        
    def get_metadata_embeddings(self) -> torch.Tensor:
        return self.metadata_embeddings.weight

    def set_metadata_embeddings(self, new_embeddings: torch.Tensor):
        self.metadata_embeddings.weight.copy_(new_embeddings)

    def get_rank_embeddings(self) -> torch.Tensor:
        return self.rank_embeddings.weight

    def set_rank_embeddings(self, new_embeddings: torch.Tensor):
        self.rank_embeddings.weight.copy_(new_embeddings)
        
    def forward(
        self,
        input_idx: torch.Tensor,
        input_embeds: Optional[torch.Tensor] = None,
        scores: Optional[torch.Tensor] = None,
        indptr: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        **kwargs,
    ) -> torch.Tensor:
        # `input_idx`: (total_input_metadata)
        # `scores`: (total_input_metadata)
        
        if input_idx is not None:
            input_idx, mask = align_tensor(input_idx, indptr, pad_tok=self.config.num_total_metadata) # (bs, num_input_metadata)
            embeds = self.metadata_embeddings(input_idx) # (bs, num_input_metadata, dim)
        else:
            assert input_embeds is not None, "Invalid input: both `input_idx` and `input_embeds` cannot be None." 
            embeds, mask = align_tensor(input_embeds, indptr)
            
        if embeds.size(1) != self.config.num_input_metadata:
            raise ValueError(
                f"Invalid input: expected {self.config.num_input_metadata} metadata items, "
                f"but got {embeds.size(1)}."
            )

        if scores is not None:
            scores, mask = align_tensor(scores, indptr) # (bs, num_input_metadata)
            
        if position_ids is None:
            position_ids = (
                self.position_ids[:, :self.config.num_input_metadata]
                if scores is None else 
                torch.argsort(scores, dim=1, descending=True)
            )
            
        rank_embeddings = self.rank_embeddings(position_ids) # (bs, num_input_metadata, dim) or (1, num_input_metadata, dim)

        embeddings = embeds + rank_embeddings

        if scores is not None:
            score_embeddings = self.score_ffn(scores.unsqueeze(2)) # (bs, num_input_metadata, dim)
            embeddings = embeddings + score_embeddings
            
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return UPMAEncoderOutput(repr=self.out_ffn(embeddings)), mask # (bs, num_input_metadata, dim), (bs, num_input_metadata)
        

#### Example

In [139]:
model = UPMAEmbeddingMemory(config)

In [141]:
inputs = {
    'input_idx': torch.randint(config.num_total_metadata, size=(5,)),
    'scores': torch.rand((5,)),
    'indptr': torch.tensor([1, 1, 0, 3], dtype=torch.int64),
}

In [143]:
embeds, mask = model(**inputs)

### `Encoder Memory`

In [19]:
#| export
class UPMAEncoderMemory(PreTrainedModel):
    config: UPMAConfig
    load_tf_weights = None
    base_model_prefix = "distilbert"
    supports_gradient_checkpointing = True
    _supports_flash_attn = True
    _supports_sdpa = True
    
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)
        self.embeddings = Embeddings(config)
        self.n_layers = config.n_memory_layers
        self.layer = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_memory_layers)])
        self.gradient_checkpointing = False

        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
        self._use_sdpa = config._attn_implementation == "sdpa"

    def _init_weights(self, module: nn.Module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        elif isinstance(module, Embeddings) and self.config.sinusoidal_pos_embds:
            create_sinusoidal_embeddings(
                self.config.max_position_embeddings, self.config.dim, module.position_embeddings.weight
            )
            
    def get_position_embeddings(self) -> nn.Embedding:
        return self.embeddings.position_embeddings

    def resize_position_embeddings(self, new_num_position_embeddings: int):
        num_position_embeds_diff = new_num_position_embeddings - self.config.max_position_embeddings

        # no resizing needs to be done if the length stays the same
        if num_position_embeds_diff == 0:
            return

        logger.info(f"Setting `config.max_position_embeddings={new_num_position_embeddings}`...")
        self.config.max_position_embeddings = new_num_position_embeddings

        old_position_embeddings_weight = self.embeddings.position_embeddings.weight.clone()

        self.embeddings.position_embeddings = nn.Embedding(self.config.max_position_embeddings, self.config.dim)

        if self.config.sinusoidal_pos_embds:
            create_sinusoidal_embeddings(
                n_pos=self.config.max_position_embeddings, dim=self.config.dim, out=self.position_embeddings.weight
            )
        else:
            with torch.no_grad():
                if num_position_embeds_diff > 0:
                    self.embeddings.position_embeddings.weight[:-num_position_embeds_diff] = nn.Parameter(
                        old_position_embeddings_weight
                    )
                else:
                    self.embeddings.position_embeddings.weight = nn.Parameter(
                        old_position_embeddings_weight[:num_position_embeds_diff]
                    )
        # move position_embeddings to correct device
        self.embeddings.position_embeddings.to(self.device)

    def get_input_embeddings(self) -> nn.Embedding:
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, new_embeddings: nn.Embedding):
        self.embeddings.word_embeddings = new_embeddings

    def _prune_heads(self, heads_to_prune: dict[int, list[list[int]]]):
        for layer, heads in heads_to_prune.items():
            self.transformer.layer[layer].attention.prune_heads(heads)

    @classmethod
    def from_pretrained(
        cls,
        config:PretrainedConfig,
    ):  
        src_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        targ_model = cls(config)
        targ_model.init_weights()
        targ_model.eval()
        
        src_sd, targ_sd = src_model.state_dict(), targ_model.state_dict()
        src_keys, targ_keys = set(src_sd.keys()), set(targ_sd.keys())
        
        for k in src_keys.intersection(targ_keys):
            assert targ_sd[k].shape == src_sd[k].shape, (
                f"Shape mismatch at key '{k}'. "
                f"Expected {targ_sd[k].shape}, but got {src_sd[k].shape} in source state_dict."
            )
            targ_sd[k].copy_(src_sd[k])

        diff_keys = targ_keys.difference(src_keys)
        transformer_keys = [k for k in src_keys if k.startswith("transformer")]
        for k in transformer_keys:
            targ_k = k.split('.', maxsplit=1)[1]
            
            if targ_k in targ_sd:
                assert targ_sd[targ_k].shape == src_sd[k].shape, (
                    f"Shape mismatch at key '{k}'. "
                    f"Expected {targ_sd[targ_k].shape}, but got {src_sd[k].shape} in source state_dict."
                )
                
                targ_sd[targ_k].copy_(src_sd[k])
                diff_keys.remove(targ_k)

        assert len(diff_keys) == 0, (
            f"Initialization error: the following parameters were not initialized in the module: {diff_keys}"
        )            
        return targ_model

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        indptr: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ) -> Union[BaseModelOutput, tuple[torch.Tensor, ...]]:
         
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            self.warn_if_padding_and_no_attention_mask(input_ids, attention_mask)
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        head_mask_is_none = head_mask is None
        # Prepare head mask if needed
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embeddings = self.embeddings(input_ids, inputs_embeds)  # (bs, seq_length, dim)
        
        def _prepare_attention_mask(attention_mask, input_shape):
            if self._use_flash_attention_2:
                attention_mask = attention_mask if (attention_mask is not None and 0 in attention_mask) else None
            else:
                if attention_mask is None:
                    attention_mask = torch.ones(input_shape, device=device)  # (bs, seq_length)
    
                if self._use_sdpa and head_mask_is_none and not output_attentions:
                    attention_mask = _prepare_4d_attention_mask_for_sdpa(
                        attention_mask, embeddings.dtype, tgt_len=input_shape[1]
                    )
            return attention_mask

        _attention_mask = _prepare_attention_mask(attention_mask, input_shape)
                
        all_hidden_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None

        hidden_state = embeddings
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_state,)
                
            layer_outputs = layer_module(
                hidden_state,
                _attention_mask,
                head_mask[i],
                output_attentions,
            )

            hidden_state = layer_outputs[-1]

            if output_attentions:
                if len(layer_outputs) != 2:
                    raise ValueError(f"The length of the layer_outputs should be 2, but it is {len(layer_outputs)}")

                attentions = layer_outputs[0]
                all_attentions = all_attentions + (attentions,)
            else:
                if len(layer_outputs) != 1:
                    raise ValueError(f"The length of the layer_outputs should be 1, but it is {len(layer_outputs)}")

        repr = Pooling.mean_pooling(hidden_state, attention_mask)
        repr, mask = align_tensor(repr, indptr, pad_tok=0.0)

        # Add last layer
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_state,)

        if not return_dict:
            return tuple(v for v in [hidden_state, all_hidden_states, all_attentions] if v is not None)
            
        output = UPMAEncoderOutput(
            repr=repr, last_hidden_state=hidden_state, hidden_states=all_hidden_states, attentions=all_attentions,
        )
        return output, mask
        

#### Example

In [141]:
kwargs = Parameters.from_aug_meta_prefix_for_feature('data', 'cat2data', **batch)
inputs = Parameters.from_data_aug_meta_prefix_for_encoder('cat2data', **kwargs)['cat2data']

In [142]:
inputs = {
    "input_ids": inputs["metadata_input_ids"],
    "attention_mask": inputs["metadata_attention_mask"],
    "indptr": inputs["metadata_data2ptr"],
}

In [143]:
model = UPMAEncoderMemory.from_pretrained(config)

In [147]:
m = DistilBertModel.from_pretrained("distilbert-base-uncased")

In [125]:
o, mask = model(**inputs)

In [166]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_scores', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_scores', 'cat2data_data2ptr', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'pcat2lbl_idx', 'pcat2lbl_lbl2ptr', 'cat2lbl_idx', 'cat2lbl_scores', 'cat2lbl_lbl2ptr', 'cat2lbl_identifier', 'cat2lbl_input_text', 'cat2lbl_input_ids', 'cat2lbl_attention_mask', 'cat2lbl_data2ptr', 'pcat2lbl_data2ptr'])

## Parameters

In [141]:
#| export
class Parameters:
    
    @staticmethod
    def from_data_aug_meta_prefix_for_encoder(prefix:str, **kwargs):
        inputs = {}
        args = [arg for arg in kwargs if prefix is not None and re.match(f'^{prefix}.*_(input_ids|attention_mask|data2ptr|idx|scores)$', arg)]
        for arg in args:
            meta,param = arg.split('_', maxsplit=1)
            inputs.setdefault(meta, {})[f"metadata_{param}"] = kwargs[arg]
        return inputs
    
    @staticmethod
    def from_aug_meta_prefix_for_feature(feat:str, prefix:str, **kwargs):
        keys = ['attention_mask', 'input_ids', 'idx', 'scores']        
        inputs = {f'{prefix}_{k}': kwargs[f'{prefix}_{k}'] for k in keys if f'{prefix}_{k}' in kwargs}
        if prefix is not None and f'{prefix}_{feat}2ptr' in kwargs:
            inputs.update({f'{prefix}_data2ptr': kwargs[f'{prefix}_{feat}2ptr']})
        return inputs

    @staticmethod
    def from_aug_meta_prefix_for_loss(feat:str, prefix:str, **kwargs):
        keys = [f'{prefix}_idx', f'p{prefix}_idx']
        args = {k: kwargs[k] for k in keys if k in kwargs}
        if prefix is not None and f'{prefix}_{feat}2ptr' in kwargs:
            args.update({f'{prefix}_data2ptr': kwargs[f'{prefix}_{feat}2ptr']})
        if prefix is not None and f'p{prefix}_{feat}2ptr' in kwargs:
            args.update({f'p{prefix}_data2ptr': kwargs[f'p{prefix}_{feat}2ptr']})

        inputs = {}
        for arg in args:
            meta,param = arg.split('_', maxsplit=1)
            inputs.setdefault(meta, {})[param] = args[arg]
        return inputs
        

### Examples

In [21]:
kwargs = Parameters.from_aug_meta_prefix_for_feature('data', 'cat2data', **batch)
kwargs.keys()

dict_keys(['cat2data_attention_mask', 'cat2data_input_ids', 'cat2data_idx', 'cat2data_data2ptr'])

In [22]:
upma_model_inputs = Parameters.from_data_aug_meta_prefix_for_encoder('cat2data', **kwargs)['cat2data']

In [23]:
upma_model_inputs.keys()

dict_keys(['metadata_attention_mask', 'metadata_input_ids', 'metadata_idx', 'metadata_data2ptr'])

## `UPMA Model`

In [24]:
#| export
class UPMAModel(PreTrainedModel):
    config: UPMAConfig
    load_tf_weights = None
    base_model_prefix = "distilbert"
    supports_gradient_checkpointing = True
    _supports_flash_attn = True
    _supports_sdpa = True
    
    def __init__(self, config: PretrainedConfig):
        super().__init__(config)
        self.embeddings = Embeddings(config)
        self.memory_modules = nn.ModuleList([get_memory_module(name).from_pretrained(config) for name in config.memory_module_names])
        
        self.n_layers = config.n_layers
        self.layer = nn.ModuleList([TransformerBlock(config) for _ in range(config.n_layers)])
        self.gradient_checkpointing = False

        self._use_flash_attention_2 = config._attn_implementation == "flash_attention_2"
        self._use_sdpa = config._attn_implementation == "sdpa"

    def _init_weights(self, module: nn.Module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        elif isinstance(module, Embeddings) and self.config.sinusoidal_pos_embds:
            create_sinusoidal_embeddings(
                self.config.max_position_embeddings, self.config.dim, module.position_embeddings.weight
            )
        elif isinstance(module, UPMAEmbeddingMemory) and self.config.sinusoidal_pos_embds:
            create_sinusoidal_embeddings(
                self.config.num_input_metadata, self.config.dim, module.rank_embeddings.weight
            )
            
    def get_position_embeddings(self) -> nn.Embedding:
        return self.embeddings.position_embeddings

    def resize_position_embeddings(self, new_num_position_embeddings: int):
        num_position_embeds_diff = new_num_position_embeddings - self.config.max_position_embeddings

        # no resizing needs to be done if the length stays the same
        if num_position_embeds_diff == 0:
            return

        logger.info(f"Setting `config.max_position_embeddings={new_num_position_embeddings}`...")
        self.config.max_position_embeddings = new_num_position_embeddings

        old_position_embeddings_weight = self.embeddings.position_embeddings.weight.clone()

        self.embeddings.position_embeddings = nn.Embedding(self.config.max_position_embeddings, self.config.dim)

        if self.config.sinusoidal_pos_embds:
            create_sinusoidal_embeddings(
                n_pos=self.config.max_position_embeddings, dim=self.config.dim, out=self.position_embeddings.weight
            )
        else:
            with torch.no_grad():
                if num_position_embeds_diff > 0:
                    self.embeddings.position_embeddings.weight[:-num_position_embeds_diff] = nn.Parameter(
                        old_position_embeddings_weight
                    )
                else:
                    self.embeddings.position_embeddings.weight = nn.Parameter(
                        old_position_embeddings_weight[:num_position_embeds_diff]
                    )
        # move position_embeddings to correct device
        self.embeddings.position_embeddings.to(self.device)

    def get_input_embeddings(self) -> nn.Embedding:
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, new_embeddings: nn.Embedding):
        self.embeddings.word_embeddings = new_embeddings

    def _prune_heads(self, heads_to_prune: dict[int, list[list[int]]]):
        for layer, heads in heads_to_prune.items():
            self.transformer.layer[layer].attention.prune_heads(heads)

    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        
        metadata_idx: Optional[torch.Tensor] = None,
        metadata_input_ids: Optional[torch.Tensor] = None,
        metadata_attention_mask: Optional[torch.Tensor] = None,
        metadata_scores: Optional[torch.Tensor] = None,
        metadata_data2ptr: Optional[torch.Tensor] = None,
        metadata_embeds: Optional[torch.Tensor] = None,
        inject_memory: Optional[bool] = True,
        
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[BaseModelOutput, tuple[torch.Tensor, ...]]:
         
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        if input_ids is not None and inputs_embeds is not None:
            raise ValueError("You cannot specify both input_ids and inputs_embeds at the same time")
        elif input_ids is not None:
            self.warn_if_padding_and_no_attention_mask(input_ids, attention_mask)
            input_shape = input_ids.size()
        elif inputs_embeds is not None:
            input_shape = inputs_embeds.size()[:-1]
        else:
            raise ValueError("You have to specify either input_ids or inputs_embeds")

        device = input_ids.device if input_ids is not None else inputs_embeds.device

        head_mask_is_none = head_mask is None
        # Prepare head mask if needed
        head_mask = self.get_head_mask(head_mask, self.config.num_hidden_layers)

        embeddings = self.embeddings(input_ids, inputs_embeds)  # (bs, seq_length, dim)
        
        if inject_memory:
            memory_embeddings, memory_mask, m_mask = [], [], attention_mask
            
            for module in self.memory_modules:
                m_embeddings, _m_mask = module(
                    input_idx = metadata_idx,
                    input_embeds = metadata_embeds,
                    scores = metadata_scores,
                    indptr = metadata_data2ptr,
                    input_ids = metadata_input_ids,
                    attention_mask = metadata_attention_mask,
                ) # (bs, num_input_metadata, dim), (bs, num_input_metadata)
                m_mask = torch.cat([m_mask, _m_mask], dim=1)
                
                memory_embeddings.append(m_embeddings.repr)
                memory_mask.append(m_mask)
            
        def _prepare_attention_mask(attention_mask, input_shape):
            if self._use_flash_attention_2:
                attention_mask = attention_mask if (attention_mask is not None and 0 in attention_mask) else None
            else:
                if attention_mask is None:
                    attention_mask = torch.ones(input_shape, device=device)  # (bs, seq_length)
    
                if self._use_sdpa and head_mask_is_none and not output_attentions:
                    attention_mask = _prepare_4d_attention_mask_for_sdpa(
                        attention_mask, embeddings.dtype, tgt_len=input_shape[1]
                    )
            return attention_mask

        attention_mask = _prepare_attention_mask(attention_mask, input_shape)
                
        all_hidden_states = () if output_hidden_states else None
        all_attentions = () if output_attentions else None

        hidden_state = embeddings
        memory_injection_ctr, n_memory_injection_layer = 0, len(self.config.memory_injection_layers)
        
        for i, layer_module in enumerate(self.layer):
            if output_hidden_states:
                all_hidden_states = all_hidden_states + (hidden_state,)

            if (
                inject_memory and memory_injection_ctr < n_memory_injection_layer and 
                i+1 == self.config.memory_injection_layers[memory_injection_ctr]
            ):
                m_embeddings, m_mask = memory_embeddings[memory_injection_ctr], memory_mask[memory_injection_ctr]
                hidden_state = torch.cat([hidden_state, m_embeddings], dim=1)
                attention_mask = _prepare_attention_mask(m_mask, m_mask.size())
                memory_injection_ctr += 1
                
            layer_outputs = layer_module(
                hidden_state,
                attention_mask,
                head_mask[i],
                output_attentions,
            )

            hidden_state = layer_outputs[-1]

            if output_attentions:
                if len(layer_outputs) != 2:
                    raise ValueError(f"The length of the layer_outputs should be 2, but it is {len(layer_outputs)}")

                attentions = layer_outputs[0]
                all_attentions = all_attentions + (attentions,)
            else:
                if len(layer_outputs) != 1:
                    raise ValueError(f"The length of the layer_outputs should be 1, but it is {len(layer_outputs)}")

        # Add last layer
        if output_hidden_states:
            all_hidden_states = all_hidden_states + (hidden_state,)

        if not return_dict:
            return tuple(v for v in [hidden_state, all_hidden_states, all_attentions] if v is not None)
            
        return BaseModelOutput(
            last_hidden_state=hidden_state, hidden_states=all_hidden_states, attentions=all_attentions
        )
        

### Example

In [25]:
config

UPMAConfig {
  "__stored_args__": {
    "apply_softmax": true,
    "calib_apply_softmax": false,
    "calib_loss_weight": 0.1,
    "calib_margin": 0.3,
    "calib_num_negatives": 10,
    "calib_tau": 0.1,
    "data_aug_meta_prefix": "cat2data",
    "data_inject_memory": true,
    "data_repr_pooling": false,
    "initialize_memory_embeddings_from_injection_layer_mean": true,
    "lbl2data_aug_meta_prefix": "cat2lbl",
    "lbl2data_inject_memory": false,
    "margin": 0.3,
    "memory_injection_layers": [
      3,
      1
    ],
    "memory_module_names": [
      "embeddings",
      "encoder"
    ],
    "metadata_dropout": 0.1,
    "metadata_embedding_file": null,
    "n_memory_layers": 3,
    "num_input_metadata": 3,
    "num_negatives": 5,
    "num_total_metadata": 656086,
    "tau": 0.1,
    "use_calib_loss": true,
    "use_encoder_parallel": false
  },
  "activation": "gelu",
  "apply_softmax": true,
  "attention_dropout": 0.1,
  "calib_apply_softmax": false,
  "calib_loss_weight": 0

In [26]:
model = UPMAModel(config)

In [27]:
upma_model_inputs.keys()

dict_keys(['metadata_attention_mask', 'metadata_input_ids', 'metadata_idx', 'metadata_data2ptr'])

In [28]:
o = model(input_ids=batch['data_input_ids'], attention_mask=batch['data_attention_mask'], 
          inject_memory=True, **upma_model_inputs)

In [30]:
o[0].shape

torch.Size([3, 38, 768])

In [384]:
def func():
    import pdb; pdb.set_trace()
    o = model(input_ids=batch['data_input_ids'], attention_mask=batch['data_attention_mask'], 
              inject_memory=True, **upma_model_inputs)
    

## `UPMAEncoder`

In [169]:
#| export
class UPMAEncoder(UPMAModel):

    @classmethod
    def from_pretrained(
        cls,
        config:PretrainedConfig,
        meta_dset:Optional[Union[MainXCDataset, SMainXCDataset]] = None,
        batch_size:Optional[int] = 100,
    ):
        src_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        targ_model = cls(config)

        targ_model.init_weights()
        targ_model.eval()
        
        src_sd, targ_sd = src_model.state_dict(), targ_model.state_dict()
        src_keys, targ_keys = set(src_sd.keys()), set(targ_sd.keys())
        
        for k in src_keys.intersection(targ_keys):
            assert targ_sd[k].shape == src_sd[k].shape, (
                f"Shape mismatch at key '{k}'. "
                f"Expected {targ_sd[k].shape}, but got {src_sd[k].shape} in source state_dict."
            )
            targ_sd[k].copy_(src_sd[k])

        diff_keys = targ_keys.difference(src_keys)
        transformer_keys = [k for k in src_keys if k.startswith("transformer")]
        for k in transformer_keys:
            targ_k = k.split('.', maxsplit=1)[1]
            
            assert targ_k in targ_sd, (
                f"Unexpected key '{targ_k}' encountered, not found in target state_dict."
            )
            
            assert targ_sd[targ_k].shape == src_sd[k].shape, (
                f"Shape mismatch at key '{k}'. "
                f"Expected {targ_sd[targ_k].shape}, but got {src_sd[k].shape} in source state_dict."
            )
            
            targ_sd[targ_k].copy_(src_sd[k])
            diff_keys.remove(targ_k)

        for module, injection_layer in zip(targ_model.memory_modules, config.memory_injection_layers):
            if config.initialize_memory_embeddings_from_injection_layer_mean and isinstance(module, UPMAEmbeddingMemory):
                meta_embeds = targ_model.initialize_memory_embeddings_from_injection_layer_mean(
                    memory_injection_layer=injection_layer-1,
                    meta_dset=meta_dset,
                    save_file=config.metadata_embedding_file,
                    batch_size=batch_size,
                    use_encoder_parallel=config.use_encoder_parallel,
                )
                with torch.no_grad():
                    module.set_metadata_embeddings(meta_embeds)
                    
        return targ_model

    def initialize_memory_embeddings_from_injection_layer_mean(
        self,
        memory_injection_layer:int,
        meta_dset:Optional[Union[MainXCDataset, SMainXCDataset]] = None,
        save_file:Optional[str] = None,
        batch_size:Optional[int] = 100,
        use_encoder_parallel:Optional[bool] = True
    ):
        if save_file is not None and os.path.exists(save_file):
            meta_embeds = torch.load(save_file)
        else:
            if meta_dset is None: 
                raise ValueError(
                    f"Invalid argument: 'meta_dset' cannot be None. "
                    f"Please pass a valid dataset."
                )
                
            meta_embeds, device = [], "cuda:0" if use_encoder_parallel else next(self.parameters()).device
            meta_dl = DataLoader(meta_dset, batch_size=batch_size, collate_fn=identity_collate_fn)
    
            model = XCDataParallel(module=self.to(device)) if use_encoder_parallel else self
            with torch.no_grad():
                for batch in tqdm(meta_dl):
                    for k, v in batch.items(): 
                        if isinstance(v, torch.Tensor): batch[k] = v.to(device)
                    output = model(**batch, data_inject_memory=False, data_output_hidden_states=True)
                    embeds = output.hidden_states[memory_injection_layer]
                    embeds = Pooling.mean_pooling(embeds, batch['data_attention_mask']).to("cpu")
                    meta_embeds.append(embeds)
                # adding pad metadata embeddings
                embeds = torch.zeros(1, embeds.shape[1], dtype=embeds.dtype, device=embeds.device)
                meta_embeds.append(embeds)
                meta_embeds = torch.cat(meta_embeds, dim=0)
                
            del output, embeds, batch
            gc.collect()
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            self.to("cpu")
            
            if save_file is not None:
                os.makedirs(os.path.dirname(save_file), exist_ok=True)
                torch.save(meta_embeds, save_file)
                
        return meta_embeds
    
    def forward(
        self, 
        data_input_ids: torch.Tensor, 
        data_attention_mask: torch.Tensor,
        data_aug_meta_prefix: Optional[str]=None,
        data_inject_memory: Optional[bool]=True,
        data_output_attentions: Optional[bool] = None,
        data_output_hidden_states: Optional[bool] = None,
        data_return_dict: Optional[bool] = None,
        **kwargs
    ):
        meta_kwargs = Parameters.from_data_aug_meta_prefix_for_encoder(data_aug_meta_prefix, **kwargs)
        meta_kwargs = meta_kwargs.get(data_aug_meta_prefix, dict())
        
        output = super().forward(
            input_ids=data_input_ids, 
            attention_mask=data_attention_mask,
            inject_memory=data_inject_memory,
            output_attentions=data_output_attentions,
            output_hidden_states=data_output_hidden_states,
            return_dict=data_return_dict,
            **meta_kwargs
        )
        
        if self.config.data_repr_pooling:
            embeds = output[0][:, :data_attention_mask.shape[1], :]
            attention_mask = data_attention_mask
        else:
            embeds = output[0]
            if 'metadata_data2ptr' in meta_kwargs and data_inject_memory:
                memory_mask = alignment_mask(meta_kwargs['metadata_data2ptr'])
                attention_mask = torch.cat([data_attention_mask] + [memory_mask for _ in range(len(self.config.memory_injection_layers))], dim=1)
            else:
                attention_mask = data_attention_mask
                
        assert embeds.shape[:2] == attention_mask.shape, (
            f"Shape mismatch: embeds.shape[:2] = {embeds.shape[:2]} "
            f"but attention_mask.shape = {attention_mask.shape}."
        )
        
        data_repr = Pooling.mean_pooling(embeds, attention_mask)
        data_repr = F.normalize(data_repr, dim=1) if config.data_normalize else data_repr
        return UPMAEncoderOutput(repr=data_repr, **output)
        

### Example

In [174]:
config = UPMAConfig(
    memory_module_names = ["embeddings", "encoder"],
    memory_injection_layers = [3, 1],
    
    num_total_metadata = block.train.dset.meta['cat_meta'].n_meta,
    num_input_metadata = 3,
    metadata_dropout = 0.1,
    
    n_memory_layers = 3,

    data_aug_meta_prefix="cat2data",
    lbl2data_aug_meta_prefix="cat2lbl",
    neg2data_aug_meta_prefix="cat2neg",

    data_inject_memory=True,
    lbl2data_inject_memory=False,
    neg2data_inject_memory=False,
    
    data_repr_pooling=True,
    data_normalize=True,

    margin=0.3,
    num_negatives=5,
    tau=0.1,
    apply_softmax=True,

    calib_margin=0.3,
    calib_num_negatives=10,
    calib_tau=0.1,
    calib_apply_softmax=False,
    
    calib_loss_weight=0.1,
    use_calib_loss=True,
    
    use_encoder_parallel=False,

    initialize_memory_embeddings_from_injection_layer_mean=False,
    metadata_embedding_file=None,
    loss_function="triplet",
)

In [171]:
meta_dset = block.train.dset.meta_dset('cat_meta')
model = UPMAEncoder.from_pretrained(config, meta_dset=meta_dset)

In [34]:
data_meta_kwargs = Parameters.from_aug_meta_prefix_for_feature('data', config.data_aug_meta_prefix, **batch)

o = model(data_input_ids=batch["data_input_ids"], data_attention_mask=batch["data_attention_mask"], 
          data_aug_meta_prefix=config.data_aug_meta_prefix, data_enrich=config.data_inject_memory, **data_meta_kwargs)

In [177]:
def func():
    import pdb; pdb.set_trace()
    o = model(data_input_ids=batch["data_input_ids"], data_attention_mask=batch["data_attention_mask"], 
          data_aug_meta_prefix=config.data_aug_meta_prefix, data_enrich=config.data_inject_memory, **data_meta_kwargs)
    

## `UPA000`

In [134]:
#| export
class UPA000(PreTrainedModel):
    use_generation, use_representation = False, True
    
    def __init__(
        self, 
        config: UPMAConfig,
        meta_dset:Optional[Union[MainXCDataset, SMainXCDataset]] = None,
        batch_size:Optional[int] = 100,
    ):
        super().__init__(config)
        self.config = config
        self.encoder = UPMAEncoder.from_pretrained(config, meta_dset=meta_dset, batch_size=batch_size)
        
        loss_kwargs = {
            'margin': config.margin, 'n_negatives': config.num_negatives, 'tau': config.tau, 
            'apply_softmax': config.apply_softmax, 'reduce': config.reduction,
        }
        self.rep_loss_fn = get_loss_function(config.loss_function)(**loss_kwargs)
        
    @classmethod
    def from_pretrained(
        cls,
        config: PretrainedConfig,
        meta_dset: Optional[Union[MainXCDataset, SMainXCDataset]] = None,
        batch_size: Optional[int] = 100,
    ):
        return cls(config, meta_dset=meta_dset, batch_size=batch_size)
        
    def forward(
        self,
        data_input_ids:Optional[torch.Tensor]=None,
        data_attention_mask:Optional[torch.Tensor]=None,
        
        lbl2data_input_ids:Optional[torch.Tensor]=None,
        lbl2data_attention_mask:Optional[torch.Tensor]=None,
        lbl2data_data2ptr:Optional[torch.Tensor]=None,
        lbl2data_idx:Optional[torch.Tensor]=None,
        lbl2data_scores:Optional[torch.Tensor]=None,

        neg2data_input_ids:Optional[torch.Tensor]=None,
        neg2data_attention_mask:Optional[torch.Tensor]=None,
        neg2data_data2ptr:Optional[torch.Tensor]=None,
        neg2data_idx:Optional[torch.Tensor]=None,
        neg2data_scores:Optional[torch.Tensor]=None,
        
        plbl2data_data2ptr:Optional[torch.Tensor]=None,
        plbl2data_idx:Optional[torch.Tensor]=None,
        
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
        **kwargs
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        encoder = XCDataParallel(module=self.encoder) if self.config.use_encoder_parallel else self.encoder
        
        data_meta_kwargs = Parameters.from_aug_meta_prefix_for_feature('data', self.config.data_aug_meta_prefix, **kwargs)
        data_o = encoder(data_input_ids=data_input_ids, data_attention_mask=data_attention_mask, 
                         data_aug_meta_prefix=self.config.data_aug_meta_prefix, data_inject_memory=self.config.data_inject_memory, 
                         data_output_hidden_states=True, **data_meta_kwargs)
        
        loss = None; lbl2data_o = neg2data_o = UPMAEncoderOutput()
        if lbl2data_input_ids is not None:
            lbl2data_meta_kwargs = Parameters.from_aug_meta_prefix_for_feature('lbl2data', self.config.lbl2data_aug_meta_prefix, **kwargs)
            lbl2data_o = encoder(data_input_ids=lbl2data_input_ids, data_attention_mask=lbl2data_attention_mask, 
                                 data_aug_meta_prefix=self.config.lbl2data_aug_meta_prefix, data_inject_memory=self.config.lbl2data_inject_memory, 
                                 data_output_hidden_states=True, **lbl2data_meta_kwargs)

            if neg2data_input_ids is not None:
                neg2data_meta_kwargs = Parameters.from_aug_meta_prefix_for_feature('neg2data', self.config.neg2data_aug_meta_prefix, **kwargs)
                neg2data_o = encoder(data_input_ids=neg2data_input_ids, data_attention_mask=neg2data_attention_mask, 
                                     data_aug_meta_prefix=self.config.neg2data_aug_meta_prefix, data_inject_memory=self.config.neg2data_inject_memory, 
                                     data_output_hidden_states=True, **neg2data_meta_kwargs)

            loss = self.rep_loss_fn(data_o.repr, pos_targ=lbl2data_o.repr, n_pos=lbl2data_data2ptr, 
                                    pos_idx=lbl2data_idx, pos_scores=lbl2data_scores,
                                    n_ppos=plbl2data_data2ptr, ppos_idx=plbl2data_idx,
                                    neg_targ=neg2data_o.repr, n_neg=neg2data_data2ptr, 
                                    neg_idx=neg2data_idx, neg_scores=neg2data_scores, **kwargs)
            
        if not return_dict:
            o = (data_o.repr, lbl2data_o.repr, neg2data_o.repr)
            return ((loss,) + o) if loss is not None else o

        return UPMAModelOutput(
            loss=loss,
            data_repr=data_o.repr,
            lbl2data_repr=lbl2data_o.repr,
            neg2data_repr=neg2data_o.repr,
        )
        

### Example

In [135]:
model = UPA000.from_pretrained(config)

In [136]:
batch.keys()

dict_keys(['data_idx', 'data_identifier', 'data_input_text', 'data_input_ids', 'data_attention_mask', 'plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_scores', 'lbl2data_data2ptr', 'lbl2data_identifier', 'lbl2data_input_text', 'lbl2data_input_ids', 'lbl2data_attention_mask', 'pcat2data_idx', 'pcat2data_data2ptr', 'cat2data_idx', 'cat2data_scores', 'cat2data_data2ptr', 'cat2data_identifier', 'cat2data_input_text', 'cat2data_input_ids', 'cat2data_attention_mask', 'pcat2lbl_idx', 'pcat2lbl_lbl2ptr', 'cat2lbl_idx', 'cat2lbl_scores', 'cat2lbl_lbl2ptr', 'cat2lbl_identifier', 'cat2lbl_input_text', 'cat2lbl_input_ids', 'cat2lbl_attention_mask', 'cat2lbl_data2ptr', 'pcat2lbl_data2ptr'])

In [137]:
o = model(**batch)

In [138]:
o.loss

tensor(0., grad_fn=<DivBackward0>)

In [139]:
def func():
    import pdb; pdb.set_trace()
    o = model(**batch)
    

In [107]:
o.data_repr, o.data_repr.shape

(tensor([[ 0.1133,  0.4918, -0.3105,  ..., -0.2339, -0.1606, -0.0092],
         [-0.1645,  0.1927, -0.7017,  ..., -0.5304,  0.1522,  0.0663],
         [-0.2392, -0.5515, -0.3255,  ...,  0.1439, -0.0111, -0.0919]],
        grad_fn=<DivBackward0>),
 torch.Size([3, 768]))

In [108]:
o.lbl2data_repr, o.lbl2data_repr.shape

(tensor([[ 0.0146,  0.5108, -0.4399,  ..., -0.3527, -0.1128, -0.0712],
         [ 0.1009,  0.3114, -0.2503,  ..., -0.2568, -0.0038,  0.0075],
         [-0.1228,  0.1947, -0.6093,  ..., -0.4707,  0.0389, -0.0159],
         [-0.1751,  0.1756, -0.6148,  ..., -0.5590,  0.1895, -0.0261],
         [-0.3955, -0.2311, -0.4383,  ..., -0.0789, -0.0377,  0.2048],
         [-0.3194, -0.6375, -0.2985,  ...,  0.1485, -0.0762,  0.2454]],
        grad_fn=<DivBackward0>),
 torch.Size([6, 768]))

In [110]:
o.neg2data_repr is None

True